### Reinitialize the file

In [1]:
# warning !! remove all except this script
## remove -i argument to run
!rm `(ls | grep -v '.ipynb')`

In [2]:
pdbid = 'dsdna'
drug  = None
jobname = f'{pdbid}_{drug}' if not drug is None else pdbid

In [3]:
import os
ff_folder  = os.path.realpath("../../../smsl/opt/force_field")
mdp_folder = os.path.realpath("../../../smsl/opt/mdp")

### Copy DNA from the file which process CHARMM

In [4]:
if drug is None:
    !cp ../../a.charmm.dna_setup/{jobname}.pgro.pdb ./
else: 
    !cp ../../b.amber.drug_ff/4.Align/{pdbid}_rotate.pgro.pdb ./{pdbid}.pgro.pdb

In [5]:
!export GMXLIB={ff_folder};\
 echo "2" | gmx pdb2gmx -f {pdbid}.pgro.pdb -o {pdbid}.gro.pdb -chainsep ter -water tip3p

                     :-) GROMACS - gmx pdb2gmx, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/dsdna/c.gromacs.run_md/1.create_system
Command line:
  gmx pdb2gmx -f dsdna.pgro.pdb -o dsdna.gro.pdb -chainsep ter -water tip3p

Select the Force Field:

From '/home/user/workfolder/g4dna_protocol/smsl/opt/force_field':

 1: AMBER14SB_parmbsc1

 2: OL21 nucleic (Zgarbova et al., J. Chem. Theory Comput. 17, 6292-6301, 2021

From '/home/user/opt/gromacs-2023/share/gromacs/top':

 3: AMBER03 protein, nucleic AMBER94 (Duan et al., J. Comp. Chem. 24, 1999-2012, 2003)

 4: AMBER94 force field (Cornell et al., JACS 117, 5179-5197, 1995)

 5: AMBER96 protein, nucleic AMBER94 (Kollman et al., Acc. Chem. Res. 29, 461-469, 1996)

 6: AMBER99 protein, nucleic AMBER94 (Wang et al., J. Comp. Chem. 21, 1049-1074, 2000)

 7: AMBER99SB protein, nucleic AMBER94 (Hornak et al., Proteins 65, 712-725, 2006

### Add box and add solvent (water)
`-d` can be change by [1.2_waterbox_setting.ipynb](1.2_waterbox_setting.ipynb)

In [6]:
!cp topol.top topol.top.bak1 

In [7]:
!cp topol.top.bak1 topol.top

In [8]:
!gmx editconf -f {jobname}.gro.pdb -o {jobname}_dode.pdb -c yes -d 0.7 -bt dodecahedron

                     :-) GROMACS - gmx editconf, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/dsdna/c.gromacs.run_md/1.create_system
Command line:
  gmx editconf -f dsdna.gro.pdb -o dsdna_dode.pdb -c yes -d 0.7 -bt dodecahedron

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 1394 atoms
No velocities found
    system size :  1.961  2.058  7.794 (nm)
    diameter    :  7.948               (nm)
    center      :  0.011  0.009  3.548 (nm)
    box vectors :  0.000  0.000  0.000 (nm)
    box angles  :   0.00   0.00   0.00 (degrees)
    box volume  :   0.00               (nm^3)
    shift       :  7.001  7.002 -0.243 (nm)
new center      :  7.011  7.011  3.305 (nm)
new box vectors :  9.348  9.348  9.348 (nm)
new box angles  :  60.00  60.00  90.00 (degrees)
new box volume  : 577.71               (nm^3)

GROMACS reminds you: "I'm a stro

### Add solvent (water)

In [9]:
!gmx solvate -cp {jobname}_dode.pdb -cs spc216.gro -o {jobname}_water.pdb -p topol.top

                     :-) GROMACS - gmx solvate, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/dsdna/c.gromacs.run_md/1.create_system
Command line:
  gmx solvate -cp dsdna_dode.pdb -cs spc216.gro -o dsdna_water.pdb -p topol.top

Reading solute configuration
Reading solvent configuration

Initialising inter-atomic distances...

         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.

NOTE: From version 5.0 gmx solvate uses the Van der Waals radii
from the source below. This means the results may be diffe

#### Add ions

## Remove PBC and See dodecahedron
select `0(System)` for output group

In [10]:
!echo "0" | gmx trjconv -s {jobname}_water.pdb -f {jobname}_water.pdb -o {jobname}_water.dode.pdb -pbc res -ur compact

                     :-) GROMACS - gmx trjconv, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/dsdna/c.gromacs.run_md/1.create_system
Command line:
  gmx trjconv -s dsdna_water.pdb -f dsdna_water.pdb -o dsdna_water.dode.pdb -pbc res -ur compact

Note that major changes are planned in future for trjconv, to improve usability and utility.
Will write pdb: Protein data bank file

         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.


         that are broken across periodic boundaries, they
         cann

In [11]:
## visualize
# !pymol {jobname}_solv.dode.pdb

### Add ions
gmx genion ~
`-conc` unit: (mol/liters) default=0.125  
sele `5(SOL)` for group of solvent molecules

In [12]:
!gmx grompp -f {mdp_folder}/ions.mdp -c {jobname}_water.pdb -p topol.top -o ions.tpr

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/dsdna/c.gromacs.run_md/1.create_system
Command line:
  gmx grompp -f /home/user/workfolder/g4dna_protocol/smsl/opt/mdp/ions.mdp -c dsdna_water.pdb -p topol.top -o ions.tpr

Ignoring obsolete mdp entry 'ns_type'

NOTE 1 [file /home/user/workfolder/g4dna_protocol/smsl/opt/mdp/ions.mdp]:
  With Verlet lists the optimal nstlist is >= 10, with GPUs >= 20. Note
  that with the Verlet scheme, nstlist has no effect on the accuracy of
  your simulation.

Setting the LD random seed to -1284640777

Generated 3916 of the 3916 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 3916 of the 3916 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'DNA'

Excluding 3 bonded neighbours molecule type 'DNA2'

Excluding 2 bonded neighbours molecule type

In [13]:
!echo "SOL" | gmx genion -s ions.tpr -o {jobname}_solv.pdb -p topol.top -pname K -nname CL -neutral -conc 0.125

                      :-) GROMACS - gmx genion, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/dsdna/c.gromacs.run_md/1.create_system
Command line:
  gmx genion -s ions.tpr -o dsdna_solv.pdb -p topol.top -pname K -nname CL -neutral -conc 0.125

Reading file ions.tpr, VERSION 2023.2 (single precision)
Reading file ions.tpr, VERSION 2023.2 (single precision)
Will try to add 85 K ions and 43 CL ions.
Select a continuous group of solvent molecules
Group     0 (         System) has 56105 elements
Group     1 (            DNA) has  1394 elements
Group     2 (          Water) has 54711 elements
Group     3 (            SOL) has 54711 elements
Group     4 (      non-Water) has  1394 elements
Select a group: Selected 3: 'SOL'
Number of (3-atomic) solvent molecules: 18237

Processing topology
Replacing 128 solute molecules in topology file (topol.top)  by 85 K and 43 CL ions.

Back Off! I 

In [14]:
## rmpbc and visualize
## check System has 12200±1 elements
!echo "0" | gmx trjconv -s {jobname}_solv.pdb -f {jobname}_solv.pdb -o {jobname}_solv.dode.pdb -pbc res -ur compact
#!pymol {jobname}_ions.dode.pdb

                     :-) GROMACS - gmx trjconv, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/dsdna/c.gromacs.run_md/1.create_system
Command line:
  gmx trjconv -s dsdna_solv.pdb -f dsdna_solv.pdb -o dsdna_solv.dode.pdb -pbc res -ur compact

Note that major changes are planned in future for trjconv, to improve usability and utility.
Will write pdb: Protein data bank file

         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.


         that are broken across periodic boundaries, they
         cannot 

### Energy Minimization

In [15]:
!gmx grompp -f {mdp_folder}/min1.mdp -c {jobname}_solv.pdb -r {jobname}_solv.pdb -p topol.top -o min1.tpr
!gmx mdrun -deffnm min1 -c min1.pdb

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/dsdna/c.gromacs.run_md/1.create_system
Command line:
  gmx grompp -f /home/user/workfolder/g4dna_protocol/smsl/opt/mdp/min1.mdp -c dsdna_solv.pdb -r dsdna_solv.pdb -p topol.top -o min1.tpr

Setting the LD random seed to -68735061

Generated 3916 of the 3916 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 3916 of the 3916 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'DNA'

Excluding 3 bonded neighbours molecule type 'DNA2'

Excluding 2 bonded neighbours molecule type 'SOL'

Excluding 1 bonded neighbours molecule type 'K'

Excluding 1 bonded neighbours molecule type 'CL'
Analysing residue names:
There are:    44        DNA residues
There are: 18109      Water residues
There are:   128        Ion residues
Analysing residues n

In [16]:
## -r argument min1.pdb or {jobname}_ions.pdb
!gmx grompp -f {mdp_folder}/min2.mdp -c min1.pdb -r {jobname}_solv.pdb -p topol.top -o min2.tpr
!gmx mdrun -deffnm min2 -c min2.pdb

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/dsdna/c.gromacs.run_md/1.create_system
Command line:
  gmx grompp -f /home/user/workfolder/g4dna_protocol/smsl/opt/mdp/min2.mdp -c min1.pdb -r dsdna_solv.pdb -p topol.top -o min2.tpr


NOTE 1 [file /home/user/workfolder/g4dna_protocol/smsl/opt/mdp/min2.mdp]:
  With Verlet lists the optimal nstlist is >= 10, with GPUs >= 20. Note
  that with the Verlet scheme, nstlist has no effect on the accuracy of
  your simulation.

Setting the LD random seed to 2144141311

Generated 3916 of the 3916 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 3916 of the 3916 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'DNA'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'DNA2'

turning H bonds into constraints..

## Make Index File

In [17]:
pdbAtom = !grep "ATOM" {pdbid}.gro.pdb
n_atom = len(pdbAtom)

In [18]:
cmd = f'''\
a 1-{n_atom}
name 9 Model
ri 1-22
name 10 ChainA
q
'''
!echo -e "{cmd}" | gmx make_ndx -f min2.pdb -o {jobname}.simulate.ndx

                     :-) GROMACS - gmx make_ndx, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/dsdna/c.gromacs.run_md/1.create_system
Command line:
  gmx make_ndx -f min2.pdb -o dsdna.simulate.ndx


Reading structure file
Going to read 0 old index file(s)
Analysing residue names:
There are:    44        DNA residues
There are: 18109      Water residues
There are:   128        Ion residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

  0 System              : 55849 atoms
  1 DNA                 :  1394 atoms
  2 K                   :    85 atoms
  3 CL                  :    43 atoms
  4 Water               : 54327 atoms
  5 SOL                 : 54327 atoms
  6 non-Water           :  1522 atoms
  7 Ion                 :   128 atoms
  8 Water_and_ions      : 54455 atoms

 nr : group      '!': not  'name' nr name   'splitch' nr    Enter: 

### Equilibrium (generate [1.3_equilibrium.slurm](1.3_equilibrium.slurm))
(In the energy minimization process, the only parameter required is the initial structure. However, for NVT or NPT processes, since the .pdb file does not contain information about speed or temperature state, an additional parameter, "-t", must be included to provide this information.)

In [19]:
slurm_script = f'''\
#!/bin/bash
#SBATCH --partition=gpu                  # Define the partition on which the job shall run
#SBATCH --job-name={jobname}                  # Job name
#SBATCH --ntasks=1                       # Total number of tasks
#SBATCH --nodes=1                        # Number of nodes on which to run
#SBATCH --cpus-per-task=20               # Threads per task
#SBATCH --gres=gpu:2                     # number of GPUs per node (gres=gpu:N)
#SBATCH --mem=50000                      # Memory per job in MB
#SBATCH --time=0-999:00:00               # Time limit ( dd-hh:mm:ss ) ← e.g. Maximum run time: 1 hour
#SBATCH --time=0-999:00:00               # Time limit ( dd-hh:mm:ss ) ← e.g. Maximum run time: 1 hour
#SBATCH --output=slurm_simulation_%j.out       # Standard output log
#SBATCH --error=slurm_simulation_%j.out        # Standard error log

  echo "Start at `date`"
  echo "Running on hosts: $SLURM_NODELIST"
  echo "Running on $SLURM_NNODES nodes."
  echo "Running $SLURM_NTASKS tasks."
  echo "Using ${{SLURM_CPUS_PER_TASK}} cpus per task"
  echo "Job id is $SLURM_JOBID"
  echo "Current working directory is `pwd`"

mdpFolder="{mdp_folder}"
mdFolder="."

## load program
export PATH=$HOME/opt/gcc-10.4.0/bin:$PATH
export LD_LIBRARY_PATH=$HOME/opt/gcc-10.4.0/lib64
source $HOME/opt/gromacs-2023/bin/GMXRC

cd ${{mdFolder}}
## HEAT
gmx grompp -f ${{mdpFolder}}/heat.mdp -c min2.pdb -r {jobname}_solv.pdb -p topol.top -o heat.tpr
gmx mdrun -deffnm heat -c heat.pdb -ntmpi 4 -nb gpu -pme gpu -npme 1 -gputasks 0011 -ntomp 6
## NVT
gmx grompp -f ${{mdpFolder}}/nvt1.mdp -c heat.pdb -r {jobname}_solv.pdb -t heat.cpt -p topol.top -o nvt1.tpr
gmx mdrun -deffnm nvt1 -c nvt1.pdb -ntmpi 4 -nb gpu -pme gpu -npme 1 -gputasks 0011 -ntomp 6
## NPT1
gmx grompp -f ${{mdpFolder}}/npt1.mdp -c nvt1.pdb -r {jobname}_solv.pdb -t nvt1.cpt -p topol.top -o npt1.tpr
gmx mdrun -deffnm npt1 -c npt1.pdb -ntmpi 4 -nb gpu -pme gpu -npme 1 -gputasks 0011 -ntomp 6
## NPT2
gmx grompp -f ${{mdpFolder}}/npt2.mdp -c npt1.pdb -r {jobname}_solv.pdb -t npt1.cpt -p topol.top -o npt2.tpr
gmx mdrun -deffnm npt2 -c npt2.pdb -ntmpi 4 -nb gpu -pme gpu -npme 1 -gputasks 0011 -ntomp 6
'''

open("1.3_equilibrium.slurm", 'w').write(slurm_script)

2162

#### Running slurm command:
```
> sbatch 1.3_equilibrium.slurm
```